In [1]:
!pip install dropconnect-tensorflow

  Created wheel for dropconnect-tensorflow: filename=dropconnect_tensorflow-0.0.2-py3-none-any.whl size=3345 sha256=e1f91b49a637c25849e6a6d1c9ddd0b377b9acf31450610733c83a14816444d3
  Stored in directory: /root/.cache/pip/wheels/41/c8/0e/a7bfd8f71f7d240e0049c1a33b10ede0f9e6d764c0a195ffd4
Successfully built dropconnect-tensorflow


In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, Input, ReLU, BatchNormalization, Flatten, MaxPool2D

from dropconnect_tensorflow import DropConnectDense, DropConnectConv2D

## Test DropConnectDense

In [3]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255

11501568/11490434 [==============================] - 0s 0us/step


In [13]:
X = tf.keras.layers.Input(shape=(784,))
x = DropConnectDense(units=128, prob=0.2, activation="relu", use_bias=True)(X)
x = DropConnectDense(units=64, prob=0.5, activation="relu", use_bias=True)(x)
y = Dense(10, activation="softmax")(x)

model = tf.keras.models.Model(X, y)

In [14]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
drop_connect_dense_2 (DropCo (None, 128)               100480    
_________________________________________________________________
drop_connect_dense_3 (DropCo (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Hyperparameters
batch_size=64
epochs=20


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

Epoch 1/20
844/844 [==============================] - 6s 4ms/step - loss: 0.7680 - accuracy: 0.8009 - val_loss: 0.2890 - val_accuracy: 0.9252
Epoch 2/20
844/844 [==============================] - 3s 3ms/step - loss: 0.2975 - accuracy: 0.9161 - val_loss: 0.2124 - val_accuracy: 0.9437
Epoch 3/20
844/844 [==============================] - 3s 3ms/step - loss: 0.2368 - accuracy: 0.9328 - val_loss: 0.1813 - val_accuracy: 0.9507
Epoch 4/20
844/844 [==============================] - 3s 3ms/step - loss: 0.2035 - accuracy: 0.9421 - val_loss: 0.1571 - val_accuracy: 0.9587
Epoch 5/20
844/844 [==============================] - 3s 3ms/step - loss: 0.1792 - accuracy: 0.9491 - val_loss: 0.1427 - val_accuracy: 0.9615
Epoch 6/20
844/844 [==============================] - 3s 3ms/step - loss: 0.1608 - accuracy: 0.9545 - val_loss: 0.1353 - val_accuracy: 0.9637
Epoch 7/20
844/844 [==============================] - 3s 3ms/step - loss: 0.1456 - accuracy: 0.9584 - val_loss: 0.1208 - val_accuracy: 0.9660
Epoch 

In [7]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

Evaluate on test data:
79/79 - 0s - loss: 0.0876 - accuracy: 0.9730
Test loss = 0.08760864287614822, Test acc: 0.9729999899864197


## Test DropConnectConv2D

In [8]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = np.expand_dims(x_train, axis=-1).astype('float32') / 255
x_test = np.expand_dims(x_test, axis=-1).astype('float32') / 255

In [9]:
X = tf.keras.layers.Input(shape=(28, 28, 1))
x = DropConnectConv2D(filters=64, kernel_size=3, strides=(1, 1), padding='valid', prob=0.1)(X)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D((2,2))(x)
x = DropConnectConv2D(filters=128, kernel_size=3, strides=(1, 1), padding='valid', prob=0.1)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D((2,2))(x)

x = Flatten()(x)
x = DropConnectDense(units=64, prob=0.3, activation="relu", use_bias=True)(x)
y = Dense(10, activation="softmax")(x)

model = tf.keras.models.Model(X, y)

In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
drop_connect_conv2d (DropCon (None, 26, 26, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 64)        256       
_________________________________________________________________
re_lu (ReLU)                 (None, 26, 26, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
drop_connect_conv2d_1 (DropC (None, 11, 11, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 128)       512 

In [11]:
# Hyperparameters
batch_size=64
epochs=10


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

Epoch 1/10
844/844 [==============================] - 35s 10ms/step - loss: 0.2375 - accuracy: 0.9303 - val_loss: 0.0647 - val_accuracy: 0.9840
Epoch 2/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0637 - accuracy: 0.9812 - val_loss: 0.0474 - val_accuracy: 0.9848
Epoch 3/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0440 - accuracy: 0.9870 - val_loss: 0.0484 - val_accuracy: 0.9862
Epoch 4/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0332 - accuracy: 0.9898 - val_loss: 0.0426 - val_accuracy: 0.9867
Epoch 5/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0256 - accuracy: 0.9921 - val_loss: 0.0351 - val_accuracy: 0.9888
Epoch 6/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0212 - accuracy: 0.9936 - val_loss: 0.0442 - val_accuracy: 0.9877
Epoch 7/10
844/844 [==============================] - 8s 10ms/step - loss: 0.0157 - accuracy: 0.9953 - val_loss: 0.0386 - val_accuracy: 0.9890
Epo

In [12]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

Evaluate on test data:
79/79 - 1s - loss: 0.0388 - accuracy: 0.9880
Test loss = 0.038835447281599045, Test acc: 0.9879999756813049
